In [1]:
import boto3
import fitz
import os

from botocore.config import Config
from dotenv import load_dotenv
from langchain_aws import ChatBedrock
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate

load_dotenv()

aws_default_region = os.getenv('AWS_DEFAULT_REGION')
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')
aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')

def get_model():
    # configuration for retries
    retry_config = Config(
        region_name='us-east-1',
        retries={
            'max_attempts': 10,
            'mode': 'standard'
        }
    )
    # initialise Bedrock runtime client
    bedrock_runtime = boto3.client("bedrock-runtime", config=retry_config)

    # model kwargs
    model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0" # adjust as needed
    # model_id ="anthropic-beta: max-tokens-3-5-sonnet-2024-07-15"
    model_kwargs = {
        "max_tokens": 8192,
        "temperature": 0.0,
        "top_k": 250,
        "top_p": 1,
        "stop_sequences": ["\n\nHuman"],
    }
    
    llm = ChatBedrock(
        client=bedrock_runtime,
        model_id=model_id,
        model_kwargs=model_kwargs,
    )
    return llm

llm = get_model()

In [2]:
def remove_after_references(text):
    # Define the substring to search for
    marker = "References"
    
    # Find the index of the substring
    index = text.find(marker)
    
    # If the substring is found, slice the string up to the index
    if index != -1:
        return text[:index + len(marker)]
    
    # If the substring is not found, return the original text
    return text

def extract_pdf_text(pdf_path):
    # Open the PDF
    doc = fitz.open(pdf_path)
    text = ""
    
    # Iterate through the pages and extract text
    for page in doc:
        text += page.get_text()
    return remove_after_references(text)

In [3]:
PDF_PATH = "pdfs/Project MUSE - Hard Choices.pdf"
pdf_text = extract_pdf_text(PDF_PATH)

In [5]:
prompt = """<system>
You are an expert business analyst with deep knowledge of project management, information systems leadership, communication, business innovation, and achieving business objectives. You have extensive experience in analyzing case studies to extract strategic, tactical, and operational insights regarding IT adoption, implementation, and organizational change.
</system>

<instructions>
Given a PDF document containing a case study, perform a thorough analysis and internalize the content. Then, use a step-by-step reasoning process to evaluate the case and solve the query provided.

Start by analyzing the details in the PDF and breaking down the information. Identify relevant strategic, operational, and contextual clues. Use these insights to weigh the options in the MCQ and eliminate incorrect ones. Finally, select the best possible answer based on the analysis.

After performing the internal reasoning, **only return the correct answer** to the query without showing your intermediate steps or thought process.

<query>
{query}
</query>

<pdf_text>
{pdf_text}
</pdf_text>
"""

prompt_template = PromptTemplate(
    template=prompt,
    input_variables=["query", "pdf_text"]
)

In [6]:
chain = prompt_template | llm | StrOutputParser()

In [20]:
QUERY = """The new CIO at KLM was 

appointed from within KLM

hired through headhunting
"""

MCQ_RESPONSE = chain.invoke({"query": QUERY, "pdf_text": pdf_text})

print(MCQ_RESPONSE)

Based on the information provided in the case study, the correct answer is:

appointed from within KLM

The case study mentions that "The EVP of the Operations Control Centre was appointed as new CIO. It was felt that having the CIO coming out of the 'real business' would help in getting the 'IT governance' discussion out of the IT area, and have it put on the business executive's agenda."

This indicates that the new CIO was appointed from within KLM, specifically from the Operations Control Centre, rather than being hired through external headhunting.
